## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly!

In [1]:
import chassisml
import os
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
import numpy as np
import getpass
import json

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

## Train model

In [3]:
digits = datasets.load_digits()
data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)

SVC(gamma=0.001)

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [4]:
def process(input_bytes):
    inputs = np.array(json.loads(input_bytes))/2
    inference_results = clf.predict(inputs)
    structured_results = []
    for inference_result in inference_results:
        structured_output = {
            "data": {
                "result": {"classPredictions": [{"class": str(inference_result), "score": str(1)}]}
            }
        }
        structured_results.append(structured_output)
    return structured_results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [5]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `process_fn` defined above

In [6]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/input_sample.json'
results = chassis_model.test(sample_filepath)
print(results)

b'[{"data":{"result":{"classPredictions":[{"class":"4","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"4","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}}]'


In [7]:
# test environment and model within Chassis service, must pass filepath here:
#test_env_result = chassis_model.test_env(sample_filepath)
#print(test_env_result)

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [7]:
response = chassis_model.publish(model_name="Sklearn Digits Docker Test",model_version="0.0.3",
                     registry_user=dockerhub_user,registry_pass=dockerhub_pass )

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [9]:
modzy_url = "https://integration.modzy.engineering"
response = chassis_model.publish(model_name="Digits Docker Test",model_version="0.0.3",
                     registry_user=dockerhub_user,registry_pass=dockerhub_pass ,
                     modzy_sample_input_path=sample_filepath,
                     modzy_api_key=modzy_api_key,modzy_url=modzy_url)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


## Run sample job using Modzy SDK
Submit inference job to our newly-deploy model running on Modzy

In [10]:
from modzy import ApiClient

client = ApiClient(base_url=f'{modzy_url}/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)

[ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "4",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "8",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "8",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "4",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "8",
          "score": "1"
        }
      ]
    }
  }
})]
